### Learning and devleloping Ranking problem on dummy data to achieve the same with the real world GItHub Data
#### Team Members
    - Omkar khanvilkar
    - Pranav Nair

# Ranking Developers For Refactoring

In [4]:
# Importing all the necessary libraries

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import nltk
import spacy
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import dask
import gensim
from gensim.models import Word2Vec
import re
import math
import sys
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn.linear_model import LogisticRegression
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor

In [5]:
#Load Raw Dataset
authors_dataset = pd.read_csv('project3-authors.csv')
authors_dataset.rename(columns={"Name":"Project"},inplace=True)
authors_dataset["Project"] = authors_dataset["Project"].astype(str).replace("adangel$pmd",1).astype(str).replace("hibernate$hibernate-validator",2).astype(str).replace("eclipse$bpmn2-modeler",3).astype(str)
authors_dataset.head()

,Project,CommitId,RefactoringType,FilePath,Class,RefactoringDetail,Message,AuthorName,AuthorEmail
0,2,a3af1683e63c9c800f486552e7e5002c24b6712b,Move Source Folder,src,NaN,Move Source Folder\tsrc to hibernate-validator...,Moved the legacy validator code into hibernate...,Hardy Ferentschik,hibernate@ferentschik.de
1,2,6d6c3deaf0b499e06077383d8d3add43f30ba099,Move Source Folder,hibernate-validator-legacy/src,NaN,Move Source Folder\thibernate-validator-legacy...,Refactored build so that compile and test can ...,Hardy Ferentschik,hibernate@ferentschik.de
2,2,6d6c3deaf0b499e06077383d8d3add43f30ba099,Move Source Folder,hibernate-validator-legacy/src/test,NaN,Move Source Folder\thibernate-validator-legacy...,Refactored build so that compile and test can ...,Hardy Ferentschik,hibernate@ferentschik.de
3,2,25350de39ef9484997999f6e6d8f00abc5d51546,Extract Variable,hibernate-validator/src/main/java/org/hibernat...,org.hibernate.validation.engine.ValidatorImpl,Extract Variable\tleafBeanInstance : Object in...,BVAL-37 ConstraintDescriptor.getLeafValue()\n\...,Emmanuel Bernard,emmanuel@hibernate.org
4,2,25350de39ef9484997999f6e6d8f00abc5d51546,Rename Method,hibernate-validator/src/main/java/org/hibernat...,org.hibernate.validation.engine.ValidatorImpl,Rename Method\tpublic getBeanConstraints() : E...,BVAL-37 ConstraintDescriptor.getLeafValue()\n\...,Emmanuel Bernard,emmanuel@hibernate.org


In [6]:
# Adding the filenames into a new feature named "file" by extracting the last element from the "FilePath" variable 
authors_dataset['file'] = ''
for idx, row in authors_dataset.iterrows():
    authors_dataset.at[idx,'file'] = authors_dataset.at[idx,'FilePath'].split('/')[-1].split('.')[0]
authors_dataset.head()

,Project,CommitId,RefactoringType,FilePath,Class,RefactoringDetail,Message,AuthorName,AuthorEmail,file
0,2,a3af1683e63c9c800f486552e7e5002c24b6712b,Move Source Folder,src,NaN,Move Source Folder\tsrc to hibernate-validator...,Moved the legacy validator code into hibernate...,Hardy Ferentschik,hibernate@ferentschik.de,src
1,2,6d6c3deaf0b499e06077383d8d3add43f30ba099,Move Source Folder,hibernate-validator-legacy/src,NaN,Move Source Folder\thibernate-validator-legacy...,Refactored build so that compile and test can ...,Hardy Ferentschik,hibernate@ferentschik.de,src
2,2,6d6c3deaf0b499e06077383d8d3add43f30ba099,Move Source Folder,hibernate-validator-legacy/src/test,NaN,Move Source Folder\thibernate-validator-legacy...,Refactored build so that compile and test can ...,Hardy Ferentschik,hibernate@ferentschik.de,test
3,2,25350de39ef9484997999f6e6d8f00abc5d51546,Extract Variable,hibernate-validator/src/main/java/org/hibernat...,org.hibernate.validation.engine.ValidatorImpl,Extract Variable\tleafBeanInstance : Object in...,BVAL-37 ConstraintDescriptor.getLeafValue()\n\...,Emmanuel Bernard,emmanuel@hibernate.org,ValidatorImpl
4,2,25350de39ef9484997999f6e6d8f00abc5d51546,Rename Method,hibernate-validator/src/main/java/org/hibernat...,org.hibernate.validation.engine.ValidatorImpl,Rename Method\tpublic getBeanConstraints() : E...,BVAL-37 ConstraintDescriptor.getLeafValue()\n\...,Emmanuel Bernard,emmanuel@hibernate.org,ValidatorImpl


In [7]:
# Splitting the data into three subsets according to the Projects

project_1_dataset = authors_dataset[authors_dataset["Project"] == '1']
project_2_dataset = authors_dataset[authors_dataset["Project"] == '2']
project_3_dataset = authors_dataset[authors_dataset["Project"] == '3']

In [8]:
# Length of each of the subset datasets
print(len(project_1_dataset))
print(len(project_2_dataset))
print(len(project_3_dataset))

8495
5399
3423


# Feature Calculation Functions

In [9]:
def tokenize_stop_words_lemmitize_vectorized_lst(ser):
    """
        Parameters: ser - list of text
        Returns: Sparse matrix vector.
        
        This function takes a block of text as input, performs tokenization, stop-word removal, lemmatization and then
        converts them into vectors using Tfidf Vectoriser
    """
    #tokenize
    tokenized_list = []
    for row in ser:
        tokenized_detail = nltk.tokenize.word_tokenize(row)
        tokenized_list.append(tokenized_detail)
    #stop words
    english_stop_words = set(nltk.corpus.stopwords.words('english'))
    tokenized_stop_words_removed = []
    for tokens in tokenized_list: 
        tokens_stop_words_removed = []
        for token in tokens:
            if token in english_stop_words:
                continue
            else:
                tokens_stop_words_removed.append(token)
        tokenized_stop_words_removed.append(tokens_stop_words_removed)
    #Lemmitize
    lemmatization = nltk.stem.WordNetLemmatizer()
    tokenized_stop_words_removed_lemmatized = []
    for tokens in tokenized_stop_words_removed:
        tokens_stop_words_removed_lemmatized = []
        for token in tokens:
            tokens_stop_words_removed_lemmatized.append(lemmatization.lemmatize(token))
        tokenized_stop_words_removed_lemmatized.append(tokens_stop_words_removed_lemmatized)
    #vectorize
    tfidf = sklearn.feature_extraction.text.TfidfVectorizer(analyzer=lambda x: x)
    vectorized = tfidf.fit_transform(tokenized_stop_words_removed_lemmatized)
    return vectorized

In [10]:
def tokenize_stop_words_lemmitize_word_to_vec(ser):
    """
        Parameters: ser - list of text
        Returns: Word-2-vec model
        
        This function takes a block of text as input, performs tokenization, stop-word removal, lemmatization and then
        trains a word-2-vec model on this text using the CBOW algorithm.
    """
    #tokenize
    tokenized_list = []
    for row in ser:
        row = re.sub('\s+',' ',row)
        tokenized_detail = nltk.tokenize.word_tokenize(row)
        tokenized_list.append(tokenized_detail)
    #stop words
    english_stop_words = set(nltk.corpus.stopwords.words('english'))
    tokenized_stop_words_removed = []
    for tokens in tokenized_list: 
        tokens_stop_words_removed = []
        for token in tokens:
            if token in english_stop_words:
                continue
            else:
                tokens_stop_words_removed.append(token)
        tokenized_stop_words_removed.append(tokens_stop_words_removed)
    #Lemmitize
    lemmatization = nltk.stem.WordNetLemmatizer()
    tokenized_stop_words_removed_lemmatized = []
    for tokens in tokenized_stop_words_removed:
        tokens_stop_words_removed_lemmatized = []
        for token in tokens:
            tokens_stop_words_removed_lemmatized.append(lemmatization.lemmatize(token))
        tokenized_stop_words_removed_lemmatized.append(tokens_stop_words_removed_lemmatized)
    #vectorize
    model = gensim.models.Word2Vec(tokenized_stop_words_removed_lemmatized, min_count = 1, size = 100, window = 5)
    return model

In [11]:
def cosine_similarity_between_refactors(vector,train_dataset,w_to_v_model):
    """
        Parameters: 
            vector - Sparse Matrix Vector representation of single Refatoring Detail instance
            train_dataset - Dataframe of the train dataset
            w_to_v_model - Word-2-vec model
            
        Returns: 
            author_scores - dictionary containing the sum, min, max and average of similarity scores between vectors using the 
                            Word Mover's distance.
                            
        
        This function takes a vector representation of the Refactoring Detail and computes the similarity score using the Word Mover's distance
        between this vector and the vector representation of every Refactoring Detail in the train dataset and computes 4 
        features - sum, min, max and average of these similarities.
        
    """
    author_refactoring_detial_cs = []
    for idx, row in train_dataset.iterrows():
        if vector == row['RefactoringDetail']:
            continue
        cs = w_to_v_model.wmdistance(vector,row['RefactoringDetail'])
        author_refactoring_detial_cs.append(cs)
    if len(author_refactoring_detial_cs) == 0:
        author_refactoring_detial_cs.append(0)
    s = sum(author_refactoring_detial_cs)
    mi = min(author_refactoring_detial_cs)
    ma = max(author_refactoring_detial_cs)
    a = np.average(author_refactoring_detial_cs)
    if s > 1000000:
        s = 1000000
    if mi > 1000000:
        mi = 1000000
    if ma > 1000000:
        ma = 1000000
    if a > 1000000:
        a = 1000000
    author_scores = {
        'total': float(s),
        'min': float(mi),
        'max': float(ma),
        'avg': float(a),
    }
    return author_scores

In [12]:
def cosine_similarity_between_messages(vector,train_dataset,w_to_v_model):
    """
        Parameters: 
            vector - Sparse Matrix Vector representation of single Refatoring Detail instance
            train_dataset - Dataframe of the train dataset
            w_to_v_model - Word-2-vec model
            
        Returns: 
            author_scores - dictionary containing the sum, min, max and average of similarity scores between vectors using the 
                            Word Mover's distance.
                            
        
        This function takes a vector representation of the Refactoring Detail and computes the similarity score using the Word Mover's distance
        between this vector and the vector representation of every Commit Message in the train dataset and computes 4 
        features - sum, min, max and average of these similarities.
        
    """
    author_refactoring_detial_cs = []
    for idx, row in train_dataset.iterrows():
        if vector == row['Message']:
            continue
        cs = w_to_v_model.wmdistance(vector,row['Message'])
        author_refactoring_detial_cs.append(cs)
    if len(author_refactoring_detial_cs) == 0:
        author_refactoring_detial_cs.append(0)
    s = sum(author_refactoring_detial_cs)
    mi = min(author_refactoring_detial_cs)
    ma = max(author_refactoring_detial_cs)
    a = np.average(author_refactoring_detial_cs)
    if s > 1000000:
        s = 1000000
    if mi > 1000000:
        mi = 1000000
    if ma > 1000000:
        ma = 1000000
    if a > 1000000:
        a = 1000000
    author_scores = {
        'total': float(s),
        'min': float(mi),
        'max': float(ma),
        'avg': float(a),
    }
    return author_scores

In [13]:
def cosine_similarity_past_files(vector,train_dataset):
    """
        Parameters: 
            vector - Sparse Matrix Vector representation of single Refatoring Detail instance
            train_dataset - Dataframe of the train dataset
            
        Returns: 
            author_scores - dictionary containing the sum of similarity scores between vectors using cosine similarity.
                            
        
        This function takes a vector representation of the Refactoring Detail and computes the similarity score
        between this vector and the vector representation of every file in the train dataset and computes the sum of these 
        similarities
        
    """
    author_scores = {}
    author_file_cs = []
    author_past_edited_files = train_dataset['file'].unique()
    for past_edited_file in author_past_edited_files:
        cur_vec = tokenize_stop_words_lemmitize_vectorized_lst([past_edited_file,vector])
        cs = cosine_similarity(cur_vec[0],cur_vec[1])
        if cs[0][0] <= 0:
            author_file_cs.append(1)
    author_scores = {
        'total': sum(author_file_cs),
    }
    return author_scores

In [14]:
def turn_raw_data_to_features(in_dataset):
    """
        Parameters: 
            in_dataset - Raw dataset
            
        Returns: 
            dataset - Dataframe of the dataset with the new features
            
        This function takes the raw dataset with the original features and uses the previous functions above to compute the
        similarities between the Refactoring Detail, commit messages and the filenames and creates a new dataset that contains
        features representing the sum, min, max and average of these similarity scores
                            
        
    """
    w_to_v_cm_model = tokenize_stop_words_lemmitize_word_to_vec(in_dataset['Message'])
    w_to_v_rd_model = tokenize_stop_words_lemmitize_word_to_vec(in_dataset['RefactoringDetail'])
    new_dataset = []
    unique_project_authors = in_dataset['AuthorName'].unique()
    grouping = 1
    for idx, row in tqdm(in_dataset.iterrows()):
        for author in unique_project_authors:
            author_data = in_dataset[in_dataset['AuthorName'] == author]
            cm_scores = cosine_similarity_between_messages(row['RefactoringDetail'],author_data,w_to_v_cm_model)
            rd_scores = cosine_similarity_between_refactors(row['RefactoringDetail'],author_data,w_to_v_rd_model)
            f_scores = cosine_similarity_past_files(row['RefactoringDetail'],author_data)
            if author == row['AuthorName']:
                data_row = [author,grouping,cm_scores['total'],cm_scores['min'],cm_scores['max'],cm_scores['avg'],rd_scores['total'],rd_scores['min'],rd_scores['max'],rd_scores['avg'],f_scores['total'],1]
            else:
                data_row = [author,grouping,cm_scores['total'],cm_scores['min'],cm_scores['max'],cm_scores['avg'],rd_scores['total'],rd_scores['min'],rd_scores['max'],rd_scores['avg'],f_scores['total'],0]
            new_dataset.append(data_row)
        grouping += 1
    scaler = StandardScaler()
    dataset = pd.DataFrame(new_dataset,columns=['Author','grouping','cm_tot','cm_min','cm_max','cm_avg','rd_tot','rd_min','rd_max','rd_avg','f_count','correct_author'])
    dataset.loc[:,'rd_min'] = scaler.fit_transform(dataset[['rd_min']])
    dataset.loc[:,'rd_max'] = scaler.fit_transform(dataset[['rd_max']])
    dataset.loc[:,'rd_avg'] = scaler.fit_transform(dataset[['rd_avg']])
    dataset.loc[:,'rd_tot'] = scaler.fit_transform(dataset[['rd_tot']])
    dataset.loc[:,'cm_min'] = scaler.fit_transform(dataset[['cm_min']])
    dataset.loc[:,'cm_max'] = scaler.fit_transform(dataset[['cm_max']])
    dataset.loc[:,'cm_avg'] = scaler.fit_transform(dataset[['cm_avg']])
    dataset.loc[:,'cm_tot'] = scaler.fit_transform(dataset[['cm_tot']])
    dataset.loc[:,'f_count'] = scaler.fit_transform(dataset[['f_count']])
    return dataset

# Model Training Functions

In [15]:
def train_model(model, prediction_function, X_train, y_train, X_test, y_test):
    """
        Parameters: 
            model : Machine Learning model
            prediction_function : Parameter representing whether to use .predict() or .predict_proba() while 
                                    predicting on the test data
            X_train : Dataframe of the training data
            y_train : Target values of the training data
            X_test : Dataframe of the test dataset
            y_test : Target values of the test dataset
            
        Returns: 
            model : Model trained on the training data.
                            
        
        This function trains a given model on the training dataset
        
    """
    
    model.fit(X_train, y_train)
    
    # y_train_pred = prediction_function(model, X_train)
 
    # print('train precision: ' + str(precision_score(y_train, y_train_pred)))
    # print('train recall: ' + str(recall_score(y_train, y_train_pred)))
    # print('train accuracy: ' + str(accuracy_score(y_train, y_train_pred)))
 
    # y_test_pred = prediction_function(model, X_test)
 
    # print('test precision: ' + str(precision_score(y_test, y_test_pred)))
    # print('test recall: ' + str(recall_score(y_test, y_test_pred)))
    # print('test accuracy: ' + str(accuracy_score(y_test, y_test_pred)))
    return model

In [17]:
def get_predicted_outcome(model, data):
    """
        Parameters:
            model : ML model
            data : Data whose output has to be predicted
            
        Returns:
            An array containing the indices of the authors with the maximum predicted probability score, for every 
            row instance in the data.
    """
    return np.argmax(model.predict_proba(data), axis=1).astype(np.float32)
 
def get_predicted_rank_logistic_regression(model, data):
    """
        Parameters:
            model : Logistic Regression model
            data : Data whose output has to be predicted
            
        Returns:
            The probability score for every row instance in the data after applying the Logistic Regression model.
    """
    return model.predict_proba(data)[:, 1]

def get_predicted_rank_random_forest(model,data):
    """
        Parameters:
            model : Random Forest model
            data : Data whose output has to be predicted
            
        Returns:
            The predicted authors for every row instance in data after applying the Randm Forest model
    """
    rslt = model.predict(data)
    return rslt

In [18]:
def build_lr_model_from_dataframe(df):
    """
        Parameters:
            df : Dataframe
            
        Returns:
            model : Logistic Regression model trained on df
            
    """
    x = []
    y = []
    author_dict = {}
    cur_auth = 1
    for idx, row in df.iterrows():
        y.append(row['correct_author'])
        if row['Author'] in author_dict:
            # x.append([author_dict[row['Author']],row['cm_tot'],row['cm_min'],row['cm_max'],row['cm_avg']])
            # x.append([author_dict[row['Author']],row['rd_tot'],row['rd_min'],row['rd_max'],row['rd_avg']])
            # x.append([author_dict[row['Author']], row['f_count']])
            x.append([author_dict[row['Author']],row['cm_tot'],row['cm_min'],row['cm_max'],row['cm_avg'],row['rd_tot'],row['rd_min'],row['rd_max'],row['rd_avg'],row['f_count']])
        else:
            # x.append([cur_auth,row['rd_tot'],row['rd_min'],row['rd_max'],row['rd_avg']])
            # x.append([cur_auth,row['cm_tot'],row['cm_min'],row['cm_max'],row['cm_avg']])
            # x.append([cur_auth,row['f_count']])
            x.append([cur_auth,row['cm_tot'],row['cm_min'],row['cm_max'],row['cm_avg'],row['rd_tot'],row['rd_min'],row['rd_max'],row['rd_avg'],row['f_count']])
            author_dict[row['Author']] = cur_auth
            cur_auth += 1   
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33)
    # rf = RandomForestRegressor(n_estimators=100, random_state=42)
    lr = LogisticRegression()
    model = train_model(lr, get_predicted_outcome, X_train, y_train, X_test, y_test)
    return model

In [19]:
def build_rf_model_from_dataframe(df):
    """
        Parameters:
            df : Dataframe
            
        Returns:
            model : Random Forest model trained on df
            
    """
    
    x = []
    y = []
    author_dict = {}
    cur_auth = 1
    for idx, row in df.iterrows():
        y.append(row['correct_author'])
        if row['Author'] in author_dict:
            # x.append([author_dict[row['Author']],row['cm_tot'],row['cm_min'],row['cm_max'],row['cm_avg']])
            # x.append([author_dict[row['Author']],row['rd_tot'],row['rd_min'],row['rd_max'],row['rd_avg']])
            # x.append([author_dict[row['Author']], row['f_count']])
            x.append([author_dict[row['Author']],row['cm_tot'],row['cm_min'],row['cm_max'],row['cm_avg'],row['rd_tot'],row['rd_min'],row['rd_max'],row['rd_avg'],row['f_count']])
        else:
            # x.append([cur_auth,row['rd_tot'],row['rd_min'],row['rd_max'],row['rd_avg']])
            # x.append([cur_auth,row['cm_tot'],row['cm_min'],row['cm_max'],row['cm_avg']])
            # x.append([cur_auth,row['f_count']])
            x.append([cur_auth,row['cm_tot'],row['cm_min'],row['cm_max'],row['cm_avg'],row['rd_tot'],row['rd_min'],row['rd_max'],row['rd_avg'],row['f_count']])
            author_dict[row['Author']] = cur_auth
            cur_auth += 1   
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.05)
    rf = RandomForestRegressor(n_estimators=100, random_state=42)
    model = train_model(rf, get_predicted_outcome, X_train, y_train, X_test, y_test)
    return model

In [20]:
def rank_developers(model,df,rank_func):
    """
        Parameters:
            model : ML model
            df : Dataframe
            rank_func : One of the three functions - get_predicted_outcome(), get_predicted_rank_logistic_regression(), 
                        get_predicted_rank_random_forest()
            
        Returns:
            ranked_authors : list
            
        This function takes an ML model, dataframe and a ranking function and applies the ML model on the dataframe, gives
        out a prediction score for the authors and then ranks these authors according to the ranking function. 
            
    """
    x = []
    y = []
    author_dict = {}
    cur_auth = 1
    for idx, row in df.iterrows():
        y.append(row['correct_author'])
        if row['Author'] in author_dict:
            # x.append([author_dict[row['Author']],row['cm_tot'],row['cm_min'],row['cm_max'],row['cm_avg']])
            # x.append([author_dict[row['Author']],row['rd_tot'],row['rd_min'],row['rd_max'],row['rd_avg']])
            # x.append([author_dict[row['Author']], row['f_count']])
            x.append([author_dict[row['Author']],row['cm_tot'],row['cm_min'],row['cm_max'],row['cm_avg'],row['rd_tot'],row['rd_min'],row['rd_max'],row['rd_avg'],row['f_count']])
        else:
            x.append([cur_auth,row['cm_tot'],row['cm_min'],row['cm_max'],row['cm_avg'],row['rd_tot'],row['rd_min'],row['rd_max'],row['rd_avg'],row['f_count']])
            author_dict[row['Author']] = cur_auth
            cur_auth += 1  
    # cols = ['Author','cm_tot','cm_min','cm_max','cm_avg']
    # cols = ['Author','rd_tot','rd_min','rd_max','rd_avg']
    # cols = ['Author','f_count']
    cols = ['Author','cm_tot','cm_min','cm_max','cm_avg','rd_tot','rd_min','rd_max','rd_avg','f_count']
    learning_data = pd.DataFrame(x,columns=cols) 
    lg_input = learning_data.astype(np.float32)
    data_with_rank = learning_data.copy()
    data_with_rank['rank'] = rank_func(model, lg_input)
    data_with_rank.sort_values(by=['rank'], inplace=True,ascending=False)
    ranked_authors = []
    a_keys = list(author_dict.keys())
    a_vals = list(author_dict.values())
    for idx, row in data_with_rank.iterrows():
        ranked_authors.append(a_keys[a_vals.index(row['Author'])])
    return ranked_authors

## Run Model Function

In [21]:
def build_results(feature_dataset,model,ranking_function):
    """
        Parameters : 
            feature_dataset : Dataframe containing all the features
            model : ML model
            ranking_function : One of the three functions - get_predicted_outcome(), get_predicted_rank_logistic_regression(), 
                        get_predicted_rank_random_forest()
        Returns :
            A list containing the Mean Reciprocal Rank and the Accuracy@K metric for K = 5
            
        This function takes a machine learning model and a dataset and calculates the Mean Reciprocal Rank and the Accuracy@K
        for the outputted predictions
    """
    Q = 0
    MRR_sum = 0
    correct = 0
    top_2 = 0
    top_3 = 0
    top_4 = 0
    top_5 = 0
    uniq_groupings = feature_dataset['grouping'].unique()
    cur_pos = 0
    for group in uniq_groupings:
        grouping_dataset = feature_dataset[feature_dataset['grouping'] == group]
        ranking = rank_developers(model,grouping_dataset,ranking_function)
        correct_author = grouping_dataset[grouping_dataset['correct_author'] == 1]
        correct_author = list(correct_author['Author'])[0]
        rank_i = (ranking.index(correct_author) + 1)
        if(rank_i == 1):
            correct += 1
            top_2 += 1
            top_3 += 1
            top_4 += 1
            top_5 += 1
        elif(rank_i == 2):
            top_2 += 1
            top_3 += 1
            top_4 += 1
            top_5 += 1
        elif(rank_i == 3):
            top_3 += 1
            top_4 += 1
            top_5 += 1
        elif(rank_i == 4):
            top_4 += 1
            top_5 += 1
        elif(rank_i == 5):
            top_5 += 1
        Q += 1
        MRR_sum += (1 / rank_i)
    MRR = (1/Q) * MRR_sum
    correct /= Q
    top_2 /=Q
    top_3 /=Q
    top_4 /=Q
    top_5 /=Q
    return [MRR,correct,top_2,top_3,top_4,top_5]

## Clean & Process Dataset

In [22]:
def clean_and_process(dataset):
    """
        This function is similar to the turn_raw_data_to_features() function described previously
    """
    w_to_v_cm_model = tokenize_stop_words_lemmitize_word_to_vec(dataset['Message'])
    w_to_v_rd_model = tokenize_stop_words_lemmitize_word_to_vec(dataset['RefactoringDetail'])
    new_dataset = []
    unique_project_authors = dataset['AuthorName'].unique()
    for idx, row in tqdm(dataset.iterrows()):
        for author in unique_project_authors:
            author_data = dataset[dataset['AuthorName'] == author]
            cm_scores = cosine_similarity_between_messages(row['RefactoringDetail'],author_data,w_to_v_cm_model)
            rd_scores = cosine_similarity_between_refactors(row['RefactoringDetail'],author_data,w_to_v_rd_model)
            f_scores = cosine_similarity_past_files(row['RefactoringDetail'],author_data)
            if author == row['AuthorName']:
                data_row = [author,cm_scores['total'],cm_scores['min'],cm_scores['max'],cm_scores['avg'],rd_scores['total'],rd_scores['min'],rd_scores['max'],rd_scores['avg'],f_scores['total'],1]
            else:
                data_row = [author,cm_scores['total'],cm_scores['min'],cm_scores['max'],cm_scores['avg'],rd_scores['total'],rd_scores['min'],rd_scores['max'],rd_scores['avg'],f_scores['total'],0]
            new_dataset.append(data_row)
    scaler = StandardScaler()
    dataset = pd.DataFrame(new_dataset,columns=['Author','cm_tot','cm_min','cm_max','cm_avg','rd_tot','rd_min','rd_max','rd_avg','f_count','correct_author'])
    dataset.loc[:,'rd_min'] = scaler.fit_transform(dataset[['rd_min']])
    dataset.loc[:,'rd_max'] = scaler.fit_transform(dataset[['rd_max']])
    dataset.loc[:,'rd_avg'] = scaler.fit_transform(dataset[['rd_avg']])
    dataset.loc[:,'rd_tot'] = scaler.fit_transform(dataset[['rd_tot']])
    dataset.loc[:,'cm_min'] = scaler.fit_transform(dataset[['cm_min']])
    dataset.loc[:,'cm_max'] = scaler.fit_transform(dataset[['cm_max']])
    dataset.loc[:,'cm_avg'] = scaler.fit_transform(dataset[['cm_avg']])
    dataset.loc[:,'cm_tot'] = scaler.fit_transform(dataset[['cm_tot']])
    dataset.loc[:,'f_count'] = scaler.fit_transform(dataset[['f_count']])
    return dataset

## Program Runners

Now we shall use all the functions defined above and apply them on the the three subsets of the original raw dataset, split previously according to the project type and calculate the MRR and the Accuracy@K for each of these three projects using Logistic Regression and Random Forest

### Project 1

In [91]:
feature_set_file = './project_1_feature_set_condensed.csv'
feature_dataset1 = pd.read_csv(feature_set_file)
feature_dataset1['Grouping'] = 0
num_authors = len(feature_dataset1['Author'].unique())
g = 1
a = 1
for idx, row in feature_dataset1.iterrows():
    feature_dataset1.at[idx,'Grouping'] = g
    if a == num_authors:
        a = 0
        g += 1
    a += 1
indexs = feature_dataset1['Grouping'].unique()
train, test = train_test_split(indexs, test_size=0.75)
train_df = feature_dataset1[feature_dataset1['grouping'].isin(train)]
test_df = feature_dataset1[feature_dataset1['grouping'].isin(test)]

In [92]:
logistic_regression_model = build_lr_model_from_dataframe(train_df)
random_forest_model = build_rf_model_from_dataframe(train_df)

In [93]:
results = [build_results(test_df,random_forest_model,get_predicted_rank_random_forest)]
results.append(build_results(test_df,logistic_regression_model,get_predicted_rank_logistic_regression))
results_df = pd.DataFrame(results,columns=['MRR','Correct','Top 2','Top 3','Top 4','Top 5'])
print("Project 1 Results")
results_df

,MRR,Correct,Top 2,Top 3,Top 4,Top 5
0,0.702698,0.594312,0.707843,0.770718,0.811597,0.838925
1,0.606169,0.447012,0.602311,0.707176,0.779605,0.832926


In [88]:
results = []
project_1_dataset.reset_index(inplace=True,drop=True)
rt_map = {}
for rt in project_1_dataset['RefactoringType'].unique():
    idxs = project_1_dataset[project_1_dataset['RefactoringType'] == rt].index
    idxs = [i for i in idxs]
    rt_map[rt] = idxs
for rt in rt_map:
    try:
        f_d = test_df[test_df['grouping'].isin(rt_map[rt])]
        result = build_results(f_d,random_forest_model,get_predicted_rank_random_forest)
        result.insert(0,rt)
        results.append(result)
    except Exception as e:
        print(e)
        continue
results_df = pd.DataFrame(results,columns=['Refactoring Type','MRR','Correct','Top 2','Top 3','Top 4','Top 5'])
print("Project 1 Results By Refacotring Type")
results_df

division by zero
division by zero


,Refactoring Type,MRR,Correct,Top 2,Top 3,Top 4,Top 5
0,Move Class,0.925000,0.866667,0.962963,0.985185,0.992593,0.992593
1,Rename Attribute,0.911905,0.842857,0.942857,1.000000,1.000000,1.000000
2,Rename Method,0.928956,0.888060,0.940299,0.970149,0.977612,0.977612
3,Extract Method,0.940196,0.901961,0.960784,0.960784,0.980392,1.000000
4,Inline Method,0.903846,0.846154,0.923077,0.923077,1.000000,1.000000
5,Rename Parameter,0.937500,0.875000,1.000000,1.000000,1.000000,1.000000
6,Extract Variable,0.800000,0.714286,0.785714,0.857143,0.857143,0.928571
7,Rename Variable,0.962302,0.940476,0.964286,0.988095,0.988095,0.988095
8,Extract Superclass,0.906250,0.875000,0.875000,0.875000,1.000000,1.000000
9,Move Method,0.907552,0.843750,0.953125,0.953125,0.968750,0.968750


### Project 2

In [35]:
feature_set_file = './project_2_feature_set_condensed.csv'
feature_dataset2 = pd.read_csv(feature_set_file)
feature_dataset2['Grouping'] = 0
num_authors = len(feature_dataset2['Author'].unique())
g = 1
a = 1
for idx, row in feature_dataset2.iterrows():
    feature_dataset2.at[idx,'Grouping'] = g
    if a == num_authors:
        a = 0
        g += 1
    a += 1
indexs = feature_dataset2['Grouping'].unique()
train, test = train_test_split(indexs, test_size=0.75)
train_df_2 = feature_dataset2[feature_dataset2['grouping'].isin(train)]
test_df_2 = feature_dataset2[feature_dataset2['grouping'].isin(test)]

In [36]:
logistic_regression_model_2 = build_lr_model_from_dataframe(train_df_2)
random_forest_model_2 = build_rf_model_from_dataframe(train_df_2)

In [37]:
results_2 = [build_results(test_df_2,random_forest_model_2,get_predicted_rank_random_forest)]
results_2.append(build_results(test_df_2,logistic_regression_model_2,get_predicted_rank_logistic_regression))
results_df_2 = pd.DataFrame(results_2,columns=['MRR','Correct','Top 2','Top 3','Top 4','Top 5'])
print('Project 2 Results')
results_df_2

Project 2 Results


,MRR,Correct,Top 2,Top 3,Top 4,Top 5
0,0.786388,0.662632,0.818457,0.903177,0.933434,0.951589
1,0.739242,0.561271,0.815431,0.921331,0.950076,0.971256


In [38]:
results_2_rt = []
project_2_dataset.reset_index(inplace=True,drop=True)
rt_map = {}
for rt in project_2_dataset['RefactoringType'].unique():
    idxs = project_2_dataset[project_2_dataset['RefactoringType'] == rt].index
    idxs = [i for i in idxs]
    rt_map[rt] = idxs
for rt in rt_map:
    try:
        f_d = test_df_2[test_df_2['grouping'].isin(rt_map[rt])]
        result = build_results(f_d,random_forest_model_2,get_predicted_rank_random_forest)
        result.insert(0,rt)
        results_2_rt.append(result)
    except Exception as e:
        print(e)
        continue
results_df_2_rt = pd.DataFrame(results_2_rt,columns=['Refactoring Type','MRR','Correct','Top 2','Top 3','Top 4','Top 5'])
print('Project 2 Results By Refactoring Type')
results_df_2_rt

division by zero
division by zero
division by zero
division by zero
division by zero
Project 2 Results By Refactoring Type


,Refactoring Type,MRR,Correct,Top 2,Top 3,Top 4,Top 5
0,Move Source Folder,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,Extract Variable,0.752778,0.583333,0.833333,0.916667,0.916667,1.000000
2,Rename Method,0.790384,0.672414,0.818966,0.905172,0.913793,0.956897
3,Move Attribute,0.802249,0.690476,0.833333,0.880952,0.952381,0.952381
4,Rename Attribute,0.866667,0.766667,0.900000,1.000000,1.000000,1.000000
5,Rename Parameter,0.867188,0.765625,0.921875,0.968750,1.000000,1.000000
6,Rename Variable,0.815891,0.662791,0.918605,0.976744,1.000000,1.000000
7,Extract And Move Method,0.833333,0.769231,0.769231,0.923077,0.923077,0.923077
8,Extract Interface,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
9,Move Method,0.708852,0.596154,0.692308,0.769231,0.807692,0.846154


### Project 3

In [39]:
feature_set_file = './project_3_feature_set.csv'
feature_dataset3 = pd.read_csv(feature_set_file)
feature_dataset3['Grouping'] = 0
num_authors = len(feature_dataset3['Author'].unique())
g = 1
a = 1
for idx, row in feature_dataset3.iterrows():
    feature_dataset3.at[idx,'Grouping'] = g
    if a == num_authors:
        a = 0
        g += 1
    a += 1
indexs = feature_dataset3['Grouping'].unique()
train, test = train_test_split(indexs, test_size=0.75)
train_3_df = feature_dataset3[feature_dataset3['grouping'].isin(train)]
test_3_df = feature_dataset3[feature_dataset3['grouping'].isin(test)]

In [40]:
logistic_regression_model_3 = build_lr_model_from_dataframe(train_3_df)
random_forest_model_3 = build_rf_model_from_dataframe(train_3_df)

In [41]:
results_3 = [build_results(test_3_df,random_forest_model_3,get_predicted_rank_random_forest)]
results_3.append(build_results(test_3_df,logistic_regression_model_3,get_predicted_rank_logistic_regression))
results_df_3 = pd.DataFrame(results_3,columns=['MRR','Correct','Top 2','Top 3','Top 4','Top 5'])
print('Project 3 Results')
results_df_3

Project 3 Results


,MRR,Correct,Top 2,Top 3,Top 4,Top 5
0,0.975136,0.959502,0.983255,0.984813,0.990654,0.997274
1,0.976817,0.957944,0.991044,0.994159,0.997274,0.998832


In [42]:
results_3_rt = []
project_3_dataset.reset_index(inplace=True,drop=True)
rt_map = {}
for rt in project_3_dataset['RefactoringType'].unique():
    idxs = project_3_dataset[project_3_dataset['RefactoringType'] == rt].index
    idxs = [i for i in idxs]
    rt_map[rt] = idxs
for rt in rt_map:
    try:
        f_d = test_3_df[test_3_df['grouping'].isin(rt_map[rt])]
        result = build_results(f_d,random_forest_model_3,get_predicted_rank_random_forest)
        result.insert(0,rt)
        results_3_rt.append(result)
    except Exception as e:
        print(e)
        continue
results_df_3_rt = pd.DataFrame(results_3_rt,columns=['Refactoring Type','MRR','Correct','Top 2','Top 3','Top 4','Top 5'])
print('Project 3 Results By Refactoring Type')
results_df_3_rt

Project 3 Results By Refactoring Type


,Refactoring Type,MRR,Correct,Top 2,Top 3,Top 4,Top 5
0,Extract Method,0.981151,0.968254,0.988095,0.988095,1.000000,1.000000
1,Extract Variable,0.958273,0.935252,0.971223,0.971223,0.971223,0.978417
2,Move And Rename Class,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
3,Move Class,0.994658,0.993590,0.993590,0.993590,0.993590,0.993590
4,Rename Variable,0.980598,0.969697,0.984848,0.984848,0.992424,0.997475
5,Rename Attribute,0.975588,0.964706,0.976471,0.976471,0.982353,1.000000
6,Rename Class,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
7,Rename Method,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
8,Move Attribute,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
9,Extract Superclass,0.961765,0.941176,0.970588,0.970588,0.970588,1.000000


### The END
